In [1]:
from scipy.spatial.transform import Rotation as R
from scipy.fft import fft, fftfreq

def rotate_accelerometer_to_world_frame(sensor_df):
    acc_data = sensor_df[['acc_x', 'acc_y', 'acc_z']].copy()
    acc_gravity = sensor_df[['acc_gx', 'acc_gy', 'acc_gz']].copy()
    rotation_rate = sensor_df[['gyro_x', 'gyro_y', 'gyro_z']].copy()

    columns = ['acc_x', 'acc_y', 'acc_z', 'acc_gx', 'acc_gy', 'acc_gz', 'mean_magnitude']
    result_df = pd.DataFrame(columns=columns)
    for acc, acc_g, gyro in zip(acc_data.itertuples(index=False), acc_gravity.itertuples(index=False), rotation_rate.itertuples(index=False)):
        roll = gyro.gyro_x
        pitch = gyro.gyro_y
        yaw = gyro.gyro_z
        r = R.from_euler('zyx', [yaw, pitch, roll], degrees=False)

        acc_world = r.apply([acc.acc_x, acc.acc_y, acc.acc_z])
        gravity_world = r.apply([acc_g.acc_gx, acc_g.acc_gy, acc_g.acc_gz])
        mean_magnitude = np.linalg.norm(acc_world)

        result = pd.DataFrame([list(acc_world) + list(gravity_world) + [mean_magnitude]], columns=columns)
        result_df = pd.concat([result_df, result], ignore_index=True)

    return result_df


def compute_frequency_domain(signal, interval):
    N = len(signal)
    fs = 1000 / interval
    frequencies = fftfreq(N, d=1/fs)
    fft_values = np.abs(fft(signal))
    
    pos_mask = frequencies > 0
    frequencies = frequencies[pos_mask]
    fft_values = fft_values[pos_mask]
    
    power = fft_values**2
    return np.sum(frequencies * power) / np.sum(power), frequencies[np.argmax(fft_values)]
    
    
def csv_preprocess(data, window_size=4000, data_interval=500):
    features = [
        'avg_acc_x', 'median_acc_x', 'std_acc_x', 'min_x', 'max_x', 'mean_abs_x',
        'avg_acc_y', 'median_acc_y', 'std_acc_y', 'min_y', 'max_y', 'mean_abs_y',
        'avg_acc_z', 'median_acc_z', 'std_acc_z', 'min_z', 'max_z', 'mean_abs_z',
        'avg_acc_gx', 'avg_acc_gy', 'avg_acc_gz',
        'gyro_z_mean', 'gyro_z_std', 'gyro_z_max', 'gyro_z_min',
        'mean_magnitude', 'signal_magnitude_area',
        'mean_freq_x', 'dominant_freq_x',
        'mean_freq_y', 'dominant_freq_y',
        'mean_freq_z', 'dominant_freq_z',
        'lat_diff', 'lon_diff',
        'label'
    ]
    result_df = pd.DataFrame(columns=features)
    
    # window sliding approach
    n_row = window_size // data_interval
    for window_start in range(0, len(data) - n_row + 1):
        window_end = window_start + n_row
        this_window = data.iloc[window_start:window_end]
        
        rotated_df = rotate_accelerometer_to_world_frame(this_window)
        
        # accelerometer of this window frame
        mean_acc_x = rotated_df['acc_x'].mean()
        median_acc_x = rotated_df['acc_x'].median()
        std_acc_x = rotated_df['acc_x'].std()
        min_acc_x = rotated_df['acc_x'].min()
        max_acc_x = rotated_df['acc_x'].max()
        mean_abs_x = rotated_df['acc_x'].abs().mean()
        # ------------------------------------
        mean_acc_y = rotated_df['acc_y'].mean()
        median_acc_y = rotated_df['acc_y'].median()
        std_acc_y = rotated_df['acc_y'].std()
        min_acc_y = rotated_df['acc_y'].min()
        max_acc_y = rotated_df['acc_y'].max()
        mean_abs_y = rotated_df['acc_y'].abs().mean()
        # ------------------------------------
        mean_acc_z = rotated_df['acc_z'].mean()
        median_acc_z = rotated_df['acc_z'].median()
        std_acc_z = rotated_df['acc_z'].std()
        min_acc_z = rotated_df['acc_z'].min()
        max_acc_z = rotated_df['acc_z'].max()
        mean_abs_z = rotated_df['acc_z'].abs().mean()
        
        # accelerometer (including gravity) of this window frame
        mean_acc_gx = rotated_df['acc_gx'].mean()
        # std_acc_gx = rotated_df['acc_gx'].std()
        # min_acc_gx = rotated_df['acc_gx'].min()
        # max_acc_gx = rotated_df['acc_gx'].max()
        # ------------------------------------
        mean_acc_gy = rotated_df['acc_gy'].mean()
        # std_acc_gy = rotated_df['acc_gy'].std()
        # min_acc_gy = rotated_df['acc_gy'].min()
        # max_acc_gy = rotated_df['acc_gy'].max()
        # ------------------------------------
        mean_acc_gz = rotated_df['acc_gz'].mean()
        # std_acc_gz = rotated_df['acc_gz'].std()
        # min_acc_gz = rotated_df['acc_gz'].min()
        # max_acc_gz = rotated_df['acc_gz'].max()
        # ------------------------------------
        gyro_z = this_window['gyro_z']
        gyro_z_mean = gyro_z.mean()
        gyro_z_std = gyro_z.std()
        gyro_z_max = gyro_z.max()
        gyro_z_min = gyro_z.min()
        
        # other features
        mean_magnitude = rotated_df['mean_magnitude'].mean()
        # ------------------------------------
        signal_magnitude_area = np.sum(np.abs([mean_acc_x, mean_acc_y, mean_acc_z]))
        # ------------------------------------
        mean_freq_x, dominant_freq_x = compute_frequency_domain(rotated_df['acc_x'], data_interval)
        mean_freq_y, dominant_freq_y = compute_frequency_domain(rotated_df['acc_y'], data_interval)
        mean_freq_z, dominant_freq_z = compute_frequency_domain(rotated_df['acc_z'], data_interval)
        # ------------------------------------
        lats = list(this_window['gps_lat'])
        lons = list(this_window['gps_lon'])
        diff_lat = lats[0] - lats[len(lats) - 1]
        diff_lon = lons[0] - lons[len(lons) - 1]
        
        label_map = {'Halt': 0, 'Forward': 1, 'Turn': 2}
        label = label_map.get(this_window['action'].mode()[0], -1)

        row = pd.DataFrame([[mean_acc_x, median_acc_x, std_acc_x, min_acc_x, max_acc_x, mean_abs_x, 
                             mean_acc_y, median_acc_y, std_acc_y, min_acc_y, max_acc_y, mean_abs_y, 
                             mean_acc_z, median_acc_z, std_acc_z, min_acc_z, max_acc_z, mean_abs_z, 
                             mean_acc_gx, mean_acc_gy, mean_acc_gz, 
                             gyro_z_mean, gyro_z_std, gyro_z_max, gyro_z_min,
                             mean_magnitude, signal_magnitude_area, 
                             mean_freq_x, dominant_freq_x, 
                             mean_freq_y, dominant_freq_y, 
                             mean_freq_z, dominant_freq_z, 
                             diff_lat, diff_lon, 
                             label]], columns=features)
        result_df = pd.concat([result_df, row], ignore_index=True)
        
    return result_df


In [2]:
import pandas as pd
import numpy as np
from keras import layers, models
from keras.callbacks import EarlyStopping

data_handHeld1 = pd.read_csv('raw_data/handheld1.csv')
data_onIpad = pd.read_csv('raw_data/handheld1.csv')
data_onTable = pd.read_csv('raw_data/handheld1.csv')
data_side1 = pd.read_csv('raw_data/handheld1.csv')
data_side2 = pd.read_csv('raw_data/handheld1.csv')

dataset = pd.concat([data_handHeld1, data_onIpad, data_onTable, data_side1, data_side2], ignore_index=True)

feature_df = csv_preprocess(dataset)

# Features and labels
X = feature_df.drop(columns=['label']).values.astype(np.float32)
y = feature_df['label'].values.astype(np.int32)

num_features = X.shape[1]
num_classes = len(np.unique(y))

print(f"Total samples: {X.shape[0]}, Features per sample: {num_features}, Classes: {num_classes}")

/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, result], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:137: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, row], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:22: FutureWarning: The behavior of DataFr

Total samples: 9003, Features per sample: 35, Classes: 3


/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, result], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, result], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_34155/3226071797.py:22: FutureWarning: The behavior of Data

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []

for train_idx, val_idx in kf.split(X):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Build NN model
    model = models.Sequential([
        layers.Input(shape=(num_features,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    # Train
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=50,
              batch_size=32,
              callbacks=[early_stop],
              verbose=0)

    # Evaluate
    val_acc = model.evaluate(X_val, y_val, verbose=0)[1]
    cv_accuracies.append(val_acc)

print("Cross-validation accuracies:", cv_accuracies)
print("Mean CV accuracy:", np.mean(cv_accuracies))

Cross-validation accuracies: [0.9983342289924622, 0.9972237348556519, 0.9983342289924622, 1.0, 0.9988889098167419]
Mean CV accuracy: 0.9985562205314636


In [4]:
input_shape = (num_features,)
print(input_shape)
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(num_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# model.save("Models/model_nn_savedmodel")  # Keras format
print("Model saved: Models/model_nn.h5")

/Users/Bay_Techatham/anaconda3/envs/Data-collecter/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


(35,)
Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.8377 - loss: 0.4654 - val_accuracy: 0.8835 - val_loss: 0.3465
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.9384 - loss: 0.1758 - val_accuracy: 0.9390 - val_loss: 0.1483
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step - accuracy: 0.9563 - loss: 0.1223 - val_accuracy: 0.9600 - val_loss: 0.1194
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - accuracy: 0.9662 - loss: 0.0917 - val_accuracy: 0.9556 - val_loss: 0.1181
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.9745 - loss: 0.0734 - val_accuracy: 0.9734 - val_loss: 0.0880
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - accuracy: 0.9803 - loss: 0.0585 - val_accuracy: 0.9811 - val_loss: 0.0557
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - accuracy: 0.9830 - loss: 0.0493 - val_accuracy: 0.9745 - val_loss: 0.0789
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.9867 - loss: 0.0

In [5]:
# model.export("Models/model_nn_savedmodel")

In [6]:
# Explicitly save in HDF5 format
# model.save("Models/model_nn_legacy.h5", include_optimizer=True)


In [7]:
# Save as TensorFlow SavedModel (compatible with TFJS)
import tensorflowjs as tfjs
model.export("Models/model_nn_savedmodel")
tfjs.converters.convert_tf_saved_model("Models/model_nn_savedmodel", "tfjs_model")


INFO:tensorflow:Assets written to: Models/model_nn_savedmodel/assets


INFO:tensorflow:Assets written to: Models/model_nn_savedmodel/assets


Saved artifact at 'Models/model_nn_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 35), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  6378945616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6378946000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6378944656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6378945424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6378945808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6378945040: TensorSpec(shape=(), dtype=tf.resource, name=None)


I0000 00:00:1757854553.201479 12266078 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1757854553.201648 12266078 single_machine.cc:374] Starting new session


In [8]:
model.save('sampleModel.h5')